In [1]:
import pandas as pd
import numpy as np
import point_cloud_utils as pcu
from plyfile import PlyData
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
# VERSION SETTING
version_dict = {
    "original" : ["kd_tree", "angular"],
    "modified" : ["01_ring", "02_xyz", "03_group8", "03_group4", "03_group2"]
}

selected_type = "modified"
version = version_dict[selected_type][2]

config_label = "test"
version = f"{version}_{config_label}"
print(f"Selected version: {version}")

Selected version: 03_group8_test


In [3]:
GEOM_BIT_DEPTH = 32 # 100m 범위 / 2cm 정밀도 기준
RING_BIT_DEPTH = 5 # 32 channels
BITS_PER_PT_RAW = (GEOM_BIT_DEPTH * 3) + 5

experiment_dir = Path("/home/noh/pgc/experiments")
INPUT_CSV = experiment_dir / "csv" / f"all_data_{version}.csv"
ORIG_DIR = Path("/home/noh/pgc/datasets/nuscenes/v1.0-mini/ply/bin")
RECON_DIR = experiment_dir / version
OUTPUT_CSV = experiment_dir / "csv" / f"final_performance_report_{version}.csv"

In [4]:
# Extract coordinates from PLY file
def load_ply_info(path):
    ply = PlyData.read(str(path))
    num_points = ply['vertex'].count
    v = np.vstack([
        ply['vertex']['x'], 
        ply['vertex']['y'], 
        ply['vertex']['z'],
    ]).T.astype(np.float64)
    return v, num_points

In [5]:
def get_peak_value(v_orig):
    # MPEG Standard: Bounding box diagonal length as peak value
    bbox_min, bbox_max = v_orig.min(axis=0), v_orig.max(axis=0)
    peak = np.linalg.norm(bbox_max - bbox_min)
    return peak

In [6]:
def calc_compression_metrics(row, num_points):
    # 1. BPP (Total and Position)
    total_bytes = row['total_bitstream_size_bytes']
    total_bits = total_bytes * 8
    pos_bytes = row['positions_bitstream_size_bytes']
    pos_bits = pos_bytes * 8
    
    # total_bpp_calc = total_bits / num_points
    log_pos_bpp = row['positions_bitstream_size_bpp']
    calc_pos_bpp = pos_bits / num_points
    
    # 2. Theoretical BPP (49 bits/pt)
    total_raw_bits = num_points * BITS_PER_PT_RAW
    total_raw_bytes = total_raw_bits / 8

    # 3. Compression Rate
    comp_rate = (1 - (total_bits / total_raw_bits)) * 100

    return total_raw_bytes, pos_bytes, log_pos_bpp, calc_pos_bpp, comp_rate

In [7]:
# MAIN
df = pd.read_csv(INPUT_CSV)
print(f"Load CSV file from: {INPUT_CSV}")
results = []
calc_count = 0

print("Start Caclualtion...")
for _, row in df.iterrows():
    scene = row['scene_name']
    f_idx = f"{int(row['frame_index']):02d}"
    orig_path = ORIG_DIR / scene / f"{scene}_{f_idx}.ply"
    recon_path = RECON_DIR / scene / f"{scene}_{f_idx}_recon.ply"
    
    # 1. Load Data and Calculate Peak value
    v_orig, num_points = load_ply_info(orig_path)
    v_recon, _ = load_ply_info(recon_path)
    peak = get_peak_value(v_orig)
    
    # 2. D1 MSE (PCU)
    sq_dists, _ = pcu.k_nearest_neighbors(v_recon, v_orig, k=1)
    mse = np.mean(sq_dists)
    psnr_d1 = 10 * np.log10((peak**2) / mse) if mse > 0 else 100.0
    
    # 3. Compresseion Metrics
    total_raw_bytes, comp_bytes, log_bpp, calc_bpp, c_rate = calc_compression_metrics(row, num_points)
    calc_count += 1
    results.append({
        'scene': scene,
        'frame': row['frame_index'],
        'original_pos_bytes': total_raw_bytes,
        'compressed_pos_bytes': comp_bytes,
        'psnr_d1': psnr_d1,
        'log_pos_bpp': log_bpp,
        'calc_pos_bpp': calc_bpp,
        'compression_rate': c_rate
        })
        
print(f"{calc_count} data are calculated")
df_final = pd.DataFrame(results).round(3)

Load CSV file from: /home/noh/pgc/experiments/csv/all_data_03_group8_test.csv
Start Caclualtion...
404 data are calculated


In [9]:
df_final.to_csv(OUTPUT_CSV, index=False)
print(f"Successfully saved to: {OUTPUT_CSV}")

Successfully saved to: /home/noh/pgc/experiments/csv/final_performance_report_03_group8_test.csv


In [8]:
df_final_avg = (
    df_final.groupby(['scene'])
    .mean(numeric_only=True)
    .drop(columns=['frame'])
    .reset_index()
    .round(3)
)
df_final_avg

,scene,original_pos_bytes,compressed_pos_bytes,psnr_d1,log_pos_bpp,calc_pos_bpp,compression_rate
0,scene-0061,438350.359,46764.462,68.474,10.775,10.775,89.313
1,scene-0103,438380.400,41822.500,67.690,9.636,9.636,90.441
2,scene-0553,438251.317,37835.878,69.111,8.720,8.720,91.348
3,scene-0655,438202.049,42294.805,67.697,9.748,9.748,90.330
4,scene-0757,438330.146,42307.122,66.954,9.748,9.748,90.330
5,scene-0796,438420.800,51041.200,68.271,11.758,11.758,88.339
6,scene-0916,438349.854,49618.854,67.624,11.433,11.433,88.662
7,scene-1077,438379.415,47785.634,67.521,11.009,11.009,89.081
8,scene-1094,438269.300,49467.900,68.702,11.400,11.400,88.695
9,scene-1100,438259.200,52122.250,69.052,12.012,12.012,88.089
